In [34]:
import tensorflow as tf
import tensorflow.keras as tfk
import tensorflow.keras.layers as tfkl
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import text, sequence
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In this example, we're going to train a [CharRNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) on a body of Shakespearian text. Ultimtely, this is an unsuperived learning task. But similar to our previous explorations in unsupervised DL, we will use an unlabeled dataset and create many samples of labeled data that we can use with our familiar supervised loss functions. The result will be a model that has learned the statistical properties of the input text, and can then be considered a "generative" model of language because we can use it to generate synthetic passages of Shakespeare.  

In [2]:
file_path = "shakespeare.txt"

with open(file_path,"r") as f:
    text = f.read()

We've loaded our Shakespeare text, let's take a look at a random snippet.

In [3]:
print(text[31600:32000])

 lies i' the second chamber?
  LADY MACBETH. Donalbain.
  MACBETH. This is a sorry sight.           [Looks on his hands.
  LADY MACBETH. A foolish thought, to say a sorry sight.
  MACBETH. There's one did laugh in 's sleep, and one cried,
      "Murther!"
    That they did wake each other. I stood and heard them,
    But they did say their prayers and address'd them
    Again to sleep.
  LADY MACB


We need to convert our text into numeric arrays, the next several blocks accomplish this.

First, we'll create a mapping between characters and their numeric index. We'll also create the reverse mapping, which is useful.

In [4]:
chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

total chars: 75


Next, we'll create a training set of sub-sequences. Remember, we're trying to train a model to be able to predict the next chracter if it is given several characters of a subsequence. So we will create training pairs where each X is a fixed-length subsequences and each Y is the corresponding next letter in the text.

In [5]:
maxlen = 40
step = 3
sub_sequences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sub_sequences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sub_sequences))

nb sequences: 38700


In [6]:
k=300
print("(Sequence):\n" + sub_sequences[k])
print("\n(Target Character): \n" + next_chars[k])

(Sequence):
 and other Apparitions
  Lords, Gentleme

(Target Character): 
n


Next we'll create one-hot vectors for our sub-sequences. The tensor we create here will be shaped as (num_sequences x sequence_length x alphabet_size).

In [7]:
X = np.zeros((len(sub_sequences), maxlen, len(chars)), dtype=np.uint8 )
Y = np.zeros((len(sub_sequences), len(chars)), dtype=np.uint8)
for i, seq in enumerate(sub_sequences):
    for t, char in enumerate(seq):
        X[i, t, char_indices[char]] = 1
        Y[i, char_indices[next_chars[i]]] = 1

In [8]:
X[0,0,:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [9]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Our RNN model will be quite simple.

In [10]:
char_rnn = Sequential()
char_rnn.add(tfkl.LSTM(128, input_shape=(maxlen, len(chars))))
char_rnn.add(tfkl.Dense(len(chars),activation="softmax"))

In [11]:
char_rnn.compile(loss='categorical_crossentropy', optimizer=tfk.optimizers.RMSprop(lr=0.01))

In [12]:
char_rnn.fit(X,Y, epochs=20, batch_size=1024)

Epoch 1/20
38/38 [==============================] - 11s 286ms/step - loss: 3.1906
Epoch 2/20
38/38 [==============================] - 11s 296ms/step - loss: 2.6179
Epoch 3/20
38/38 [==============================] - 13s 332ms/step - loss: 2.2126
Epoch 4/20
38/38 [==============================] - 12s 328ms/step - loss: 1.9993
Epoch 5/20
38/38 [==============================] - 12s 311ms/step - loss: 1.8661
Epoch 6/20
38/38 [==============================] - 12s 308ms/step - loss: 1.7657
Epoch 7/20
38/38 [==============================] - 12s 312ms/step - loss: 1.6779
Epoch 8/20
38/38 [==============================] - 11s 296ms/step - loss: 1.6042
Epoch 9/20
38/38 [==============================] - 11s 298ms/step - loss: 1.5431
Epoch 10/20
38/38 [==============================] - 11s 296ms/step - loss: 1.4907
Epoch 11/20
38/38 [==============================] - 11s 296ms/step - loss: 1.4385
Epoch 12/20
38/38 [==============================] - 11s 293ms/step - loss: 1.3926
Epoch 13/20
3

Once we have a trained model, we can simulate new text by making predictions about the next character and then drawing characters in proportion to the predicted probabilities. And then simple repeat that process over and over, each time drawing the next character.

In [13]:
def draw_char(probs):
    probs = np.asarray(probs).astype('float64')
    if sum(probs) != 1.0:
        probs = probs / np.sum(probs)
    draw = np.random.choice(range(len(probs)) , p=probs)
    return draw

def sample_text(model, sample_length=100):
    start = np.random.randint(0, len(text) - maxlen - 1)
    sequence = text[start: start + maxlen]
  
    x_preds = np.zeros((sample_length, maxlen, len(chars)))
    for i in range(sample_length):
        for t, char in enumerate(sequence[-maxlen:]):
            x_preds[i, t, char_indices[char]] = 1.

        preds = model.predict(np.expand_dims(x_preds[i,:,:], axis=0), verbose=0)[0]
        next_index = draw_char(preds)
        next_char = indices_char[next_index]

        sequence += next_char
    return sequence

In [14]:
sim = sample_text(char_rnn,sample_length=500) 

In [15]:
print(sim)

slaughter on their souls. Heaven rest thes
    When il I with onfel -hourds no mice, and fill of villoon,
    What's weremorance as a poost soverver,
    The porny the gids burne. Wheles conthin, mainst atcomparts to not gition
    What wall keepon's groon to by hay look!
    Sheme a sevil be the camal of Copthe (on.
    That's caumsion call, thee, I day you prooine
    What nawed no kelfore they laegd at thes; lignk
    That nest stie, your's him leave in may Head,
    And thee castlemonive -ighles notnoursing
    What our caustabes 


Notice that we can do pretty well to learn the typical statistical patterns of this text and then simulate new text that appears to be very similar to legitimate Shakespeare. 

But just a caution - we can also do pretty well with a much simpler method (Markov model): http://nbviewer.jupyter.org/gist/yoavg/d76121dfde2618422139

So the lesson is to try something simple before jumping right in to deep learning.

## Exercise

In this example, we're going to use an RNN for sequence classification. The task we'll set up is to generate a training set of randomized strings, and train our model to detect whether a string contains any vowels.

First, we'll create a training dataset of short randomized character sequences and the corresponding label of whether or not they contain at least one vowel.

In [18]:
import string

In [19]:
def contains_vowels(sequence):
    vowels = ["a", "e", "i", "o", "u"]
    return any([vowel in list(sequence) for vowel in vowels])

In [20]:
contains_vowels("gradient")

True

In [21]:
sequences = []
labels = []
for i in range(1000):
    char_list = np.random.choice( list(string.ascii_lowercase), size = 5, replace=True)
    seq = "".join(char_list)
    sequences.append(seq)
    labels.append(int(contains_vowels(seq)))

In [22]:
df = pd.DataFrame({"sequence": sequences, "label":labels})

In [23]:
df.head()

,sequence,label
0,qcxds,0
1,fjzbm,0
2,aiyos,1
3,wqkbl,0
4,nqktg,0


Next, set up and train an RNN (of any type) to solve this task. What preprocessing will you need to do first on the raw data in order to prepare it for the network?

In [28]:
# Data Preprocessing
chars = string.ascii_lowercase
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

MAX_NB_WORDS = len(chars)
MAX_SEQUENCE_LENGTH = 5
# Lets do letter embedding, instead of one-hot ;)
EMBEDDING_DIM = 10

tokenizer = text.Tokenizer(num_words=MAX_NB_WORDS, char_level=True)
tokenizer.fit_on_texts(df['sequence'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

total chars: 26
Found 26 unique tokens.


In [33]:
X = tokenizer.texts_to_sequences(df['sequence'].values)
X = sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Y = pd.get_dummies(df['label']).values
print('Shape of label tensor:', Y.shape)

Shape of data tensor: (1000, 5)
Shape of label tensor: (1000, 2)


In [36]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print('Train tensor shape:', X_train.shape,Y_train.shape)
print('Test tensor shape:', X_test.shape,Y_test.shape)

Train tensor shape: (800, 5) (800, 2)
Test tensor shape: (200, 5) (200, 2)


In [38]:
from tensorflow.keras.callbacks import EarlyStopping

In [43]:
# Model setup and training
model = Sequential()
model.add(tfkl.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(tfkl.LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(tfkl.Dense(2, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 20
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, 
                    batch_size=batch_size,validation_split=0.1,
                    callbacks=[EarlyStopping(monitor='val_loss', patience=2, min_delta=0.0001)])

Epoch 1/20
12/12 [==============================] - 0s 30ms/step - loss: 0.6779 - accuracy: 0.6361 - val_loss: 0.6326 - val_accuracy: 0.7125
Epoch 2/20
12/12 [==============================] - 0s 10ms/step - loss: 0.6415 - accuracy: 0.6361 - val_loss: 0.5697 - val_accuracy: 0.7125
Epoch 3/20
12/12 [==============================] - 0s 9ms/step - loss: 0.6092 - accuracy: 0.6361 - val_loss: 0.5204 - val_accuracy: 0.7125
Epoch 4/20
12/12 [==============================] - 0s 9ms/step - loss: 0.5047 - accuracy: 0.7028 - val_loss: 0.3488 - val_accuracy: 0.8750
Epoch 5/20
12/12 [==============================] - 0s 9ms/step - loss: 0.3700 - accuracy: 0.8403 - val_loss: 0.2676 - val_accuracy: 0.9000
Epoch 6/20
12/12 [==============================] - 0s 9ms/step - loss: 0.2995 - accuracy: 0.8722 - val_loss: 0.2284 - val_accuracy: 0.9000
Epoch 7/20
12/12 [==============================] - 0s 10ms/step - loss: 0.2510 - accuracy: 0.9139 - val_loss: 0.1948 - val_accuracy: 0.9625
Epoch 8/20
12/12 

In [45]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5, 10)             260       
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               71168     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 71,686
Trainable params: 71,686
Non-trainable params: 0
_________________________________________________________________


In [44]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.4f}\n  Accuracy: {:0.4f}'.format(accr[0],accr[1]))

7/7 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Test set
  Loss: 0.0031
  Accuracy: 1.0000
